In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')/255.0
        self.test_x_f1 = np.load('../testImage_canny.npy')/255.0
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')/255.0
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')/255.0

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.x_f1: batch_x_f1_test,
                                                                   self.x_f2: batch_x_f2_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 7.17, total 50 epochs ptime: 378.63
#  [*] Training finished!
#  [*] Best Epoch:  20 , Accuracy:  0.8299999833106995
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-20
#  [*] Finished testing Best Epoch: 20 , accuracy:  0.8300000429153442 

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32

Epoch: [ 5] [  12/  20] time: 37.6889, loss: 0.00453151
Epoch: [ 5] [  13/  20] time: 38.0250, loss: 0.03966414
Epoch: [ 5] [  14/  20] time: 38.3604, loss: 0.02931387
Epoch: [ 5] [  15/  20] time: 38.6958, loss: 0.03004435
Epoch: [ 5] [  16/  20] time: 39.0319, loss: 0.08508901
Epoch: [ 5] [  17/  20] time: 39.3694, loss: 0.03179241
Epoch: [ 5] [  18/  20] time: 39.7081, loss: 0.02818531
Epoch: [ 5] [  19/  20] time: 40.0422, loss: 0.04777849
[5/50] - ptime: 7.1125 loss: 0.03286054 acc: 0.77000 lr: 0.00100000
Epoch: [ 6] [   0/  20] time: 41.1564, loss: 0.01692673
Epoch: [ 6] [   1/  20] time: 41.4901, loss: 0.04090896
Epoch: [ 6] [   2/  20] time: 41.8233, loss: 0.03011173
Epoch: [ 6] [   3/  20] time: 42.1575, loss: 0.00601389
Epoch: [ 6] [   4/  20] time: 42.4950, loss: 0.02554172
Epoch: [ 6] [   5/  20] time: 42.8285, loss: 0.00733499
Epoch: [ 6] [   6/  20] time: 43.1621, loss: 0.02366106
Epoch: [ 6] [   7/  20] time: 43.4977, loss: 0.01493681
Epoch: [ 6] [   8/  20] time: 43.837

Epoch: [12] [  10/  20] time: 89.2617, loss: 0.00281210
Epoch: [12] [  11/  20] time: 89.5986, loss: 0.00801758
Epoch: [12] [  12/  20] time: 89.9357, loss: 0.00355978
Epoch: [12] [  13/  20] time: 90.2715, loss: 0.00188788
Epoch: [12] [  14/  20] time: 90.6080, loss: 0.00197522
Epoch: [12] [  15/  20] time: 90.9447, loss: 0.00642548
Epoch: [12] [  16/  20] time: 91.2799, loss: 0.00914538
Epoch: [12] [  17/  20] time: 91.6149, loss: 0.01293946
Epoch: [12] [  18/  20] time: 91.9541, loss: 0.00634168
Epoch: [12] [  19/  20] time: 92.2907, loss: 0.00096497
[12/50] - ptime: 7.0986 loss: 0.00601562 acc: 0.74000 lr: 0.00090000
Epoch: [13] [   0/  20] time: 93.4831, loss: 0.00040206
Epoch: [13] [   1/  20] time: 93.8185, loss: 0.00068967
Epoch: [13] [   2/  20] time: 94.1549, loss: 0.00683210
Epoch: [13] [   3/  20] time: 94.4927, loss: 0.01050522
Epoch: [13] [   4/  20] time: 94.8282, loss: 0.00130877
Epoch: [13] [   5/  20] time: 95.1626, loss: 0.00008885
Epoch: [13] [   6/  20] time: 95.50

Epoch: [19] [   6/  20] time: 140.4102, loss: 0.00077214
Epoch: [19] [   7/  20] time: 140.7453, loss: 0.00983752
Epoch: [19] [   8/  20] time: 141.0790, loss: 0.00001053
Epoch: [19] [   9/  20] time: 141.4168, loss: 0.00006146
Epoch: [19] [  10/  20] time: 141.7522, loss: 0.00615398
Epoch: [19] [  11/  20] time: 142.0867, loss: 0.00011059
Epoch: [19] [  12/  20] time: 142.4224, loss: 0.00467236
Epoch: [19] [  13/  20] time: 142.7573, loss: 0.00635715
Epoch: [19] [  14/  20] time: 143.0914, loss: 0.00030420
Epoch: [19] [  15/  20] time: 143.4270, loss: 0.00012844
Epoch: [19] [  16/  20] time: 143.7628, loss: 0.00491975
Epoch: [19] [  17/  20] time: 144.1000, loss: 0.00048010
Epoch: [19] [  18/  20] time: 144.4352, loss: 0.00732113
Epoch: [19] [  19/  20] time: 144.7699, loss: 0.00002232
[19/50] - ptime: 7.0895 loss: 0.00364012 acc: 0.73000 lr: 0.00090000
Epoch: [20] [   0/  20] time: 145.8866, loss: 0.00057597
Epoch: [20] [   1/  20] time: 146.2213, loss: 0.00000169
Epoch: [20] [   2/ 

Epoch: [26] [   2/  20] time: 191.5484, loss: 0.00017027
Epoch: [26] [   3/  20] time: 191.8894, loss: 0.00000808
Epoch: [26] [   4/  20] time: 192.2256, loss: 0.00007737
Epoch: [26] [   5/  20] time: 192.5629, loss: 0.00000529
Epoch: [26] [   6/  20] time: 192.9036, loss: 0.00006404
Epoch: [26] [   7/  20] time: 193.2419, loss: 0.00000975
Epoch: [26] [   8/  20] time: 193.5821, loss: 0.00002714
Epoch: [26] [   9/  20] time: 193.9213, loss: 0.00680270
Epoch: [26] [  10/  20] time: 194.2569, loss: 0.00001658
Epoch: [26] [  11/  20] time: 194.5938, loss: 0.00010682
Epoch: [26] [  12/  20] time: 194.9330, loss: 0.00018072
Epoch: [26] [  13/  20] time: 195.2708, loss: 0.00004112
Epoch: [26] [  14/  20] time: 195.6086, loss: 0.00004043
Epoch: [26] [  15/  20] time: 195.9471, loss: 0.00017414
Epoch: [26] [  16/  20] time: 196.2828, loss: 0.00000790
Epoch: [26] [  17/  20] time: 196.6178, loss: 0.00051876
Epoch: [26] [  18/  20] time: 196.9567, loss: 0.00002327
Epoch: [26] [  19/  20] time: 1

Epoch: [32] [  19/  20] time: 242.3711, loss: 0.00003057
[32/50] - ptime: 7.1383 loss: 0.00007298 acc: 0.66000 lr: 0.00072900
Epoch: [33] [   0/  20] time: 243.6000, loss: 0.00019698
Epoch: [33] [   1/  20] time: 243.9369, loss: 0.00000246
Epoch: [33] [   2/  20] time: 244.2733, loss: 0.00000248
Epoch: [33] [   3/  20] time: 244.6105, loss: 0.00000226
Epoch: [33] [   4/  20] time: 244.9488, loss: 0.00001718
Epoch: [33] [   5/  20] time: 245.2856, loss: 0.00000725
Epoch: [33] [   6/  20] time: 245.6212, loss: 0.00010228
Epoch: [33] [   7/  20] time: 245.9587, loss: 0.00003822
Epoch: [33] [   8/  20] time: 246.2971, loss: 0.00007250
Epoch: [33] [   9/  20] time: 246.6346, loss: 0.00003213
Epoch: [33] [  10/  20] time: 246.9726, loss: 0.00007955
Epoch: [33] [  11/  20] time: 247.3142, loss: 0.00001702
Epoch: [33] [  12/  20] time: 247.6533, loss: 0.00002714
Epoch: [33] [  13/  20] time: 247.9922, loss: 0.00005580
Epoch: [33] [  14/  20] time: 248.3290, loss: 0.00000810
Epoch: [33] [  15/ 

Epoch: [39] [  15/  20] time: 293.6551, loss: 0.00000944
Epoch: [39] [  16/  20] time: 293.9921, loss: 0.00000265
Epoch: [39] [  17/  20] time: 294.3278, loss: 0.00005959
Epoch: [39] [  18/  20] time: 294.6630, loss: 0.00000393
Epoch: [39] [  19/  20] time: 295.0009, loss: 0.00002201
[39/50] - ptime: 7.0890 loss: 0.00001505 acc: 0.69000 lr: 0.00072900
Epoch: [40] [   0/  20] time: 296.0611, loss: 0.00000362
Epoch: [40] [   1/  20] time: 296.3964, loss: 0.00001472
Epoch: [40] [   2/  20] time: 296.7342, loss: 0.00000062
Epoch: [40] [   3/  20] time: 297.0693, loss: 0.00000064
Epoch: [40] [   4/  20] time: 297.4062, loss: 0.00000066
Epoch: [40] [   5/  20] time: 297.7454, loss: 0.00000101
Epoch: [40] [   6/  20] time: 298.0840, loss: 0.00001406
Epoch: [40] [   7/  20] time: 298.4216, loss: 0.00000018
Epoch: [40] [   8/  20] time: 298.7581, loss: 0.00000256
Epoch: [40] [   9/  20] time: 299.0941, loss: 0.00005298
Epoch: [40] [  10/  20] time: 299.4335, loss: 0.00002630
Epoch: [40] [  11/ 

Epoch: [46] [  11/  20] time: 345.0155, loss: 0.00000364
Epoch: [46] [  12/  20] time: 345.3536, loss: 0.00000120
Epoch: [46] [  13/  20] time: 345.6908, loss: 0.00002526
Epoch: [46] [  14/  20] time: 346.0299, loss: 0.00002295
Epoch: [46] [  15/  20] time: 346.3672, loss: 0.00000071
Epoch: [46] [  16/  20] time: 346.7046, loss: 0.00001324
Epoch: [46] [  17/  20] time: 347.0440, loss: 0.00000455
Epoch: [46] [  18/  20] time: 347.3838, loss: 0.00000098
Epoch: [46] [  19/  20] time: 347.7201, loss: 0.00000138
[46/50] - ptime: 7.1978 loss: 0.00000856 acc: 0.69000 lr: 0.00065610
Epoch: [47] [   0/  20] time: 348.8535, loss: 0.00000197
Epoch: [47] [   1/  20] time: 349.1903, loss: 0.00000904
Epoch: [47] [   2/  20] time: 349.5299, loss: 0.00000220
Epoch: [47] [   3/  20] time: 349.8700, loss: 0.00001244
Epoch: [47] [   4/  20] time: 350.2076, loss: 0.00000063
Epoch: [47] [   5/  20] time: 350.5480, loss: 0.00061438
Epoch: [47] [   6/  20] time: 350.8864, loss: 0.00000111
Epoch: [47] [   7/ 

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放